- tensorflow: 딥러닝 프레임워크
- tensorflow-hub: 사전 학습된 모델을 쉽게 불러오는 라이브러리
- opencv-python: 카메라 영상 처리 및 시각화
- pandas: 라벨 데이터(객체 이름)를 읽어오기 위해 필요

In [1]:
!pip install tensorflow tensorflow-hub opencv-python numpy pandas

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 13.1 MB/s  0:00:00

   ---------------------------------------- 0/2 [tf-keras]
   ---------------------------------------- 0/2 [tf-keras]
   ---------------------------------------- 0/2 [tf-keras]
   ---------------------------------------- 0/2 [tf-keras]
   ---------------------------------------- 0/2 [tf-keras]
   ---------------------------------------- 0/2 [tf-keras]
   ---------------------------------------- 0/2 [tf-keras]
   ---------------------------------------- 0/2 [tf-keras]
   ---------------------------------------- 0/2 [tf-keras]
   ---------------------------------------- 0/2 [tf-keras]
   ---------------------------------------- 0/2 [tf-keras]
   ---------------------------------------- 0/2 [tf-keras]
   ---------------------------------------- 0/2 [tf-keras]
   ---------------------------------------- 0/2 [tf-keras]
   ---------------------------

In [2]:
import cv2
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import time

# 1. 모델 로드(tensorflow hub에서 SSD MobileNet V2 다운로드)
print("모델을 로드하는 중입니다... 잠시 기다려주세요.")
model_handle = "https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2"
detector = hub.load(model_handle)
print("모델 로드 완료!")


모델을 로드하는 중입니다... 잠시 기다려주세요.



모델 로드 완료!


In [3]:
# 2. 라벨(Class Nmaes) 로드 함수
# COCO 데이터셋의 라벨(사람, 차, 컵 등 90개 클래스)을 가려옵니다.
def load_labels():
    labels_path = tf.keras.utils.get_file(
        'coco_labels.csv',
        'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/mscoco_label_map.pbtxt'
    )
    # 90개 클래스를매핑하는 리스트를 직접 정의(오류 방지용)
    # 실제 COCO 데이터셋의 주요 객체들
    labels = {
        1: 'Person', 2: 'Bicycle', 3: 'Car', 4: 'Motorcycle', 5: 'Airplane',
        6: 'Bus', 7: 'Train', 8: 'Truck', 10: 'Traffic light', 11: 'Fire hydrant',
        13: 'Stop sign', 15: 'Bench', 16: 'Bird', 17: 'Cat', 18: 'Dog',
        27: 'Backpack', 28: 'Umbrella', 31: 'Handbag', 32: 'Tie', 33: 'Suitcase',
        44: 'Bottle', 46: 'Wine glass', 47: 'Cup', 48: 'Fork', 49: 'Knife',
        50: 'Spoon', 51: 'Bowl', 52: 'Banana', 53: 'Apple', 54: 'Sandwich',
        55: 'Orange', 56: 'Broccoli', 57: 'Carrot', 58: 'Hot dog', 59: 'Pizza',
        60: 'Donut', 61: 'Cake', 62: 'Chair', 63: 'Couch', 64: 'Potted plant',
        65: 'Bed', 67: 'Dining table', 70: 'Toilet', 72: 'TV', 73: 'Laptop',
        74: 'Mouse', 75: 'Remote', 76: 'Keyboard', 77: 'Cell phone', 84: 'Book'
    }
    return labels

classes = load_labels()

5056/5056 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step


In [5]:
# 3. 웹캠 설정
cap = cv2.VideoCapture(0) # 0번은 기본 카메라
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

print("실시간 탐지를 시작합니다. 종료하려면 'q'를 누르세요.")

while True:
    start_time = time.time()
    
    # 프레임 읽기
    ret, frame = cap.read()
    if not ret:
        break

    # 4. 이미지 전처리 (TensorFlow 모델 입력 형식에 맞춤)
    # OpenCV는 BGR로 읽지만, TensorFlow는 RGB를 원함
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # 이미지를 텐서로 변환 및 차원 추가 (Height, Width, 3) -> (1, Height, Width, 3)
    input_tensor = tf.convert_to_tensor(rgb_frame)
    input_tensor = input_tensor[tf.newaxis, ...]

    # 타입 변환 (uint8 -> uint8 그대로 사용 가능)
    
    # 5. 추론 (Inference) 수행
    result = detector(input_tensor)

    # 결과 추출
    result = {key: value.numpy() for key, value in result.items()}
    
    boxes = result['detection_boxes'][0] # 바운딩 박스 좌표 [y_min, x_min, y_max, x_max]
    scores = result['detection_scores'][0] # 정확도 (0~1)
    class_ids = result['detection_classes'][0].astype(int) # 객체 ID

    # 6. 결과 시각화 (Bounding Box 그리기)
    height, width, _ = frame.shape
    
    for i in range(len(scores)):
        score = scores[i]
        
        # 신뢰도가 50% 이상인 객체만 표시
        if score > 0.5:
            box = boxes[i]
            ymin, xmin, ymax, xmax = box
            
            # 좌표를 픽셀 단위로 변환
            left = int(xmin * width)
            top = int(ymin * height)
            right = int(xmax * width)
            bottom = int(ymax * height)

            # 클래스 이름 가져오기
            class_name = classes.get(class_ids[i], 'Unknown')
            
            # 박스와 텍스트 그리기
            label = f"{class_name}: {score*100:.1f}%"
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            
            # 텍스트 배경 (가독성을 위해)
            label_size, base_line = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            top = max(top, label_size[1])
            cv2.rectangle(frame, (left, top - label_size[1]), (left + label_size[0], top + base_line), (0, 255, 0), cv2.FILLED)
            cv2.putText(frame, label, (left, top), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

    # FPS 계산 및 표시
    fps = 1.0 / (time.time() - start_time)
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # 7. 화면 출력
    cv2.imshow('TensorFlow Real-time Detection', frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

실시간 탐지를 시작합니다. 종료하려면 'q'를 누르세요.
